In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from matplotlib import pyplot as plt
from sklearn import metrics
%matplotlib inline

Lendo arquivo CSV

In [9]:
df = pd.read_csv('emprestimo.csv', sep=';')
print(df.head())

  montante   idade salario tem_conta empresta
0    Medio  Senior   Baixo       Sim      Nao
1    Medio  Senior   Baixo       Nao      Nao
2    Baixo  Senior   Baixo       Sim      Sim
3     Alto   Media   Baixo       Sim      Sim
4     Alto   Jovem    Alto       Sim      Sim


Nomeando dados

In [10]:
translate = {
    'montante': { 'Alto': 2, 'Medio': 1, 'Baixo': 0 },
    'idade': { 'Senior': 2, 'Media': 1, 'Jovem': 0 },
    'salario': { 'Alto': 1, 'Baixo': 0 },
    'tem_conta': { 'Sim': 1, 'Nao': 0 },
    'empresta': { 'Sim': 1, 'Nao': 0 }
}

for col in df:
    df[col] = df[col].apply(lambda k: translate[col][k])
print(df.head())
#o codigo não pode ser rodado 2x.

   montante  idade  salario  tem_conta  empresta
0         1      2        0          1         0
1         1      2        0          0         0
2         0      2        0          1         1
3         2      1        0          1         1
4         2      0        1          1         1


Definindo Variaveis e resultados

In [11]:
x = df[['montante', 'idade', 'salario', 'tem_conta']]
y = df['empresta']
print(y.head())

0    0
1    0
2    1
3    1
4    1
Name: empresta, dtype: int64


Importando Train test split

In [14]:
from sklearn.model_selection import train_test_split
ttp = train_test_split

Definindo SEED

In [15]:
SEED = 7

Separando casos de testes e treino

In [17]:
treino_x, teste_x, treino_y, teste_y = ttp(x, y, random_state=SEED, test_size=0.25, stratify= y )
print(teste_x.shape)
print(teste_y.shape)

(4, 4)
(4,)


In [18]:
from dataclasses import dataclass
@dataclass(frozen=True)
class ModelResult:
    n: int
    clf: DecisionTreeClassifier
    acc: float

In [20]:
def tree_model(max_depth: int):
    # clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42) # usa "gini" como padrão
    # constrói o modelo usando critério “Entropia” e max_depth variável
    clf = DecisionTreeClassifier(max_depth=max_depth, criterion='entropy', random_state=42) 
    
    # treina o modelo – observe que nessa abordagem está usando 33% para testes (ver acima!)
    # (sem separação)
    clf.fit(treino_x, treino_y)

    # aplica o modelo e mede acurácia
    pred_y = clf.predict(teste_x)
    acc = metrics.accuracy_score(teste_y, pred_y)
    return ModelResult(acc=acc, clf=clf, n=max_depth)

# mostra resultados de acurácil para cada modelo gerado por tree_model
results = {n: tree_model(n) for n in range(1,4)}
print(*[(n, r.acc) for n, r in results.items()])

(1, 0.75) (2, 0.75) (3, 1.0)
